In [2]:
!pip install pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from datetime import datetime
import re

warnings.filterwarnings('ignore')

In [3]:
# prompt: mount this folder

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Cover letter data set (hugging face)



In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/TalentFit/cover-letter-train.csv"
df = pd.read_csv(file_path)

##Cleaning
1. repeated
2. missing

In [ ]:
df.describe()
#813 rows

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
count,813,811,813,813,813,812,806,805,813
unique,56,540,159,263,345,378,482,215,760
top,Data Scientist,PhD in Computer Science,XYZ Analytics,John Smith,Data Analyst at ABC Company,Data Scientist at DEF Company,"Python, R, SQL, Machine Learning, Deep Learning",Bachelor's degree in Computer Science,I am writing to express my interest in the Sen...
freq,249,33,84,365,125,56,35,56,4


In [ ]:
missing_values_count = df.isnull().sum()
print(missing_values_count, "\n")

Job Title                     0
Preferred Qualifications      2
Hiring Company                0
Applicant Name                0
Past Working Experience       0
Current Working Experience    1
Skillsets                     7
Qualifications                8
Cover Letter                  0
dtype: int64 



In [ ]:
df_cleaned = df.dropna(inplace=False)
#there was 16 rows with empty values

In [ ]:
duplicate_rows = df_cleaned[df_cleaned.duplicated()]
duplicate_rows.head()

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
208,Data Scientist,\n 1. Analyzing large datasets to identify t...,XYZ Analytics,John Smith,Data Analyst at ABC Corporation,Data Scientist at DEF Company,"Hard skills: Python, R, SQL, machine learning...",Master's degree in Statistics,"Dear Hiring Manager,\n\nI am writing to expres..."
695,Network Engineer,"CCNA or CCNP certification, understanding of n...",Network Services Corp.,James Anderson,Network Support Engineer at NetLink for 4 years,Network Engineer at Network Services for 1 year,"Network Protocols, CCNA, CCNP, Troubleshooting",Bachelor's degree in Network Engineering,I am writing to express my interest in the Net...


In [ ]:
df_cleaned.drop_duplicates(inplace=True)
# there was 2 duplicated entries

In [ ]:
df_cleaned.info()
df_cleaned.describe()
#final no of entries = 794

<class 'pandas.core.frame.DataFrame'>
Index: 794 entries, 0 to 812
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Job Title                   794 non-null    object
 1   Preferred Qualifications    794 non-null    object
 2   Hiring Company              794 non-null    object
 3   Applicant Name              794 non-null    object
 4   Past Working Experience     794 non-null    object
 5   Current Working Experience  794 non-null    object
 6   Skillsets                   794 non-null    object
 7   Qualifications              794 non-null    object
 8   Cover Letter                794 non-null    object
dtypes: object(9)
memory usage: 62.0+ KB


,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
count,794,794,794,794,794,794,794,794,794
unique,56,527,159,263,342,376,472,215,743
top,Data Scientist,PhD in Computer Science,XYZ Analytics,John Smith,Data Analyst at ABC Company,Data Scientist at DEF Company,"Python, R, SQL, Machine Learning, Deep Learning",Bachelor's degree in Computer Science,I am eager to bring my expertise in machine le...
freq,239,33,76,347,119,52,35,55,4


##Abstracting individual skills


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def clean_skills(skills):
    # Use the API to extract and reformat skills
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": """
        Given a description of skills, rephrase them to list only the main skill keywords,
        correct any typos, remove duplicates, and exclude any unnecessary words.
        The output should only contain simplified, single-word skills or short phrases where necessary.
        Output should be a comma seperated list in the following format: Skill 1, Skill 2, Skill 3, ...
        Convert any descriptions of experience or qualities into relevant skill keywords.
        Remove any adjectives unless they are part of a skill's standard terminology.
        """
        },
        {"role": "user", "content":skills}
      ],
      max_tokens=512
    )
    return response.choices[0].message.content

In [ ]:
df_cleaned['skills'] = df_cleaned['Skillsets'].apply(clean_skills)

In [ ]:
df_cleaned.head()

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter,skills
0,Senior Java Developer,5+ years of experience in Java Development,Google,John Doe,Java Developer at XYZ for 3 years,Senior Java Developer at ABC for 2 years,"Java, Spring Boot, Hibernate, SQL",BSc in Computer Science,I am writing to express my interest in the Sen...,"Java, Spring Boot, Hibernate, SQL"
1,Data Scientist,5 years of experience in data modeling,XYZ Analytics Solutions,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Technologies,"Data modeling, data analysis, programming (Pyt...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres...","Data modeling, data analysis, programming, Pyt..."
2,Data Scientist,Experience with Python and proficiency in at l...,XYZ Analytics,John Smith,Data Analyst at ABC Corporation,Data Scientist at XYZ Solutions,"Python, R, SQL, Machine Learning, Statistical ...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres...","Python, R, SQL, Machine Learning, Statistics, ..."
3,Senior Data Scientist,Minimum of 3 years validated experience\nOutst...,XYZ Tech Solutions,John Smith,Data Analyst at ABC Company,Senior Data Scientist at DEF Corporation,"Python, machine learning libraries, deep learn...","Bachelor's degree in Computer Science, Master'...","Dear Hiring Manager,\n\nI am writing to expres...","Python, machine learning, deep learning, SQL, ..."
4,data scientist,35 years+ professional experience in data sci...,xyz tech solutions,john smith,data analyst at abc company,data scientist at xyz tech solutions,"Python, R, Machine Learning, NLP, Data Visuali...",35 years+ professional experience in data sci...,"Dear Hiring Manager,\n\nI am writing to expres...","Python, R, Machine Learning, NLP, Data Visuali..."


In [ ]:
df_cleaned.to_csv('/content/drive/MyDrive/Colab Notebooks/TalentFit/cover-letter-train-cleaned.csv')


In [48]:
file_path2 = '/content/drive/MyDrive/Colab Notebooks/TalentFit/final_coverletter_df_train.csv'
cleaned_df = pd.read_csv(file_path2)

In [51]:
cleaned_df['skills'] = cleaned_df['skills'].str.lower()
cleaned_df['skills'] = cleaned_df['skills'].str.replace('-', ' ')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('apis', 'api')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('c/c++', 'c, c++')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('business intelligence.', 'business intelligence')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('cloud.', 'cloud')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('data science.', 'data science')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('fast paced.', 'fast paced')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('teamwork.', 'teamwork')
cleaned_df['skills'] = cleaned_df['skills'].str.replace( 'problemsolving','problem solving')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('r programming', 'r')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('r)', 'r')
cleaned_df['skills'] = cleaned_df['skills'].str.replace(' ml', 'machine learning')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('machine learningops', 'mlops')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('machine learningflow', 'mlflow')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('scikitlearn', 'scikit-learn')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('sklearn', 'scikit-learn')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('powerbi', 'power bi')
cleaned_df['skills'] = cleaned_df['skills'].str.replace(' ms,', 'microsoft')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('ms office', 'microsoft office suite')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('programming (python', 'python')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('python programming', 'python')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('analysis', 'analytics')
cleaned_df['skills'] = cleaned_df['skills'].str.replace( 'organizational.','organizational')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('organization','organizational')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('organizationalal','organizational')
cleaned_df['skills'] = cleaned_df['skills'].str.replace('research.','research')

In [52]:
unique_skill = set()
for skills in cleaned_df['skills']:
  for skill in skills.split(','):
    unique_skill.add(skill.strip())


print(f"Number of unique skills: {len(unique_skill)}")
#Number of unique skills: 487
unique_skill

Number of unique skills: 487


{'*nix',
 '.net',
 'a/b testing',
 'active directory',
 'adaptability',
 'adf',
 'adobe analytics',
 'advanced analytics',
 'advertising optimization',
 'agile',
 'agile development.',
 'agricultural systems',
 'ai',
 'airflow',
 'algorithms',
 'amplitude',
 'analytical',
 'analytical skills',
 'analytical thinking',
 'analytics',
 'android',
 'ansible',
 'anti money laundering',
 'apache airflow',
 'apache hadoop',
 'apache kafka',
 'apache spark',
 'api',
 'api development',
 'api expertise',
 'arcgis',
 'arch linux',
 'architecture',
 'artificial intelligence',
 'asynchronous',
 'attention to detail',
 'automation',
 'automation technologies',
 'automation testing',
 'aws',
 'aws quicksight',
 'aws sagemaker',
 'azure',
 'azure data lake',
 'azure databricks',
 'basel ii',
 'bash',
 'bgp',
 'bi tools',
 'bi/visualization tools',
 'bidding',
 'big data',
 'big data platforms',
 'big data technologies',
 'bigquery',
 'bioinformatics',
 'biometrics',
 'blackarch',
 'business forecastin

In [53]:
cleaned_df.to_csv('/content/drive/MyDrive/Colab Notebooks/TalentFit/final_coverletter_df_train.csv')

##Test dataset

In [ ]:
file_path_test = "/content/drive/MyDrive/Colab Notebooks/TalentFit/cover-letter.csv"
df_test = pd.read_csv(file_path_test)

In [ ]:
df_test.describe()
#349 rows

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
count,349,348,349,349,349,349,345,346,349
unique,32,246,98,152,198,221,228,123,339
top,Data Scientist,PhD in AI,XYZ Analytics,John Smith,Data Analyst at ABC Company,Data Scientist at XYZ Solutions,"Python, R, SQL, Machine Learning, Deep Learning",Bachelor's degree in Computer Science,I am writing to apply for the AI Engineer posi...
freq,97,14,35,144,40,15,11,30,3


###Clean

In [ ]:
missing_values_count = df_test.isnull().sum()
print(missing_values_count, "\n")

Job Title                     0
Preferred Qualifications      1
Hiring Company                0
Applicant Name                0
Past Working Experience       0
Current Working Experience    0
Skillsets                     4
Qualifications                3
Cover Letter                  0
dtype: int64 



In [ ]:
df_cleaned_test = df_test.dropna(inplace=False)
#7 rows with missing info

In [ ]:
duplicate_rows = df_cleaned_test[df_cleaned_test.duplicated()]
duplicate_rows.head()
#no duplicates

,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter


In [ ]:
df_cleaned_test.info()
df_cleaned_test.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 341 entries, 0 to 348
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Job Title                   341 non-null    object
 1   Preferred Qualifications    341 non-null    object
 2   Hiring Company              341 non-null    object
 3   Applicant Name              341 non-null    object
 4   Past Working Experience     341 non-null    object
 5   Current Working Experience  341 non-null    object
 6   Skillsets                   341 non-null    object
 7   Qualifications              341 non-null    object
 8   Cover Letter                341 non-null    object
dtypes: object(9)
memory usage: 26.6+ KB


,Job Title,Preferred Qualifications,Hiring Company,Applicant Name,Past Working Experience,Current Working Experience,Skillsets,Qualifications,Cover Letter
count,341,341,341,341,341,341,341,341,341
unique,30,239,98,152,196,218,224,120,331
top,Data Scientist,PhD in AI,XYZ Analytics,John Smith,Data Analyst at ABC Company,Data Scientist at XYZ Solutions,"Python, R, SQL, Machine Learning, Deep Learning",Bachelor's degree in Computer Science,I am writing to apply for the AI Engineer posi...
freq,93,14,31,136,40,15,11,29,3


##Abstraction

In [ ]:
df_cleaned_test['skills'] = df_cleaned_test['Skillsets'].apply(clean_skills)

In [ ]:
df_cleaned_test.to_csv('/content/drive/MyDrive/Colab Notebooks/TalentFit/cover-letter-test-cleaned.csv')


In [4]:
file_path_test2 = '/content/drive/MyDrive/Colab Notebooks/TalentFit/cover-letter-test-cleaned.csv'
final_test_df = pd.read_csv(file_path_test2)

In [5]:
final_test_df['skills'] = final_test_df['skills'].str.lower()
final_test_df['skills'] = final_test_df['skills'].str.replace('problem-solving', 'problem solving')
final_test_df['skills'] = final_test_df['skills'].str.replace('c/c++', 'c, c++')
final_test_df['skills'] = final_test_df['skills'].str.replace('communication.', 'communication')
final_test_df['skills'] = final_test_df['skills'].str.replace('sql servers', 'sql server')
final_test_df['skills'] = final_test_df['skills'].str.replace('fast-paced', 'fast paced')
final_test_df['skills'] = final_test_df['skills'].str.replace('problemsolving', 'problem solving')
final_test_df['skills'] = final_test_df['skills'].str.replace('analysis', 'analytics')
final_test_df['skills'] = final_test_df['skills'].str.replace('r)', 'r')
final_test_df['skills'] = final_test_df['skills'].str.replace(' ml', ' machine learning')
final_test_df['skills'] = final_test_df['skills'].str.replace('powerbi', 'power bi')
final_test_df['skills'] = final_test_df['skills'].str.replace('machine learninglib', 'mllib')


In [6]:
unique_skill_test = set()
for skills in final_test_df['skills']:
  for skill in skills.split(','):
    unique_skill_test.add(skill.strip())


print(f"Number of unique skills: {len(unique_skill_test)}")
#Number of unique skills: 280
unique_skill_test

Number of unique skills: 282


{'a/b testing',
 'active directory',
 'adaptability',
 'agile',
 'ai',
 'algorithms',
 'alteryx',
 'amazon sagemaker',
 'analytical',
 'analytical skills',
 'analytical thinking',
 'analytics',
 'android',
 'angularjs',
 'apache cassandra',
 'apache spark',
 'api development',
 'artificial intelligence',
 'attention to detail',
 'automation',
 'aws',
 'aws athena',
 'aws sagemaker',
 'backbox',
 'big data',
 'big data analytics',
 'bigquery',
 'bilingual',
 'blockchain',
 'business analytics',
 'business development',
 'business intelligence',
 'business intelligence software',
 'c',
 'c#',
 'c++',
 'ccie',
 'ccna',
 'ccnp',
 'cfengine',
 'ci/cd',
 'cisco',
 'cloud',
 'cloud analytics',
 'cloud computing',
 'cloud data warehouses',
 'cloud platforms',
 'cloud technologies',
 'clustering',
 'coaching',
 'collaboration',
 'communication',
 'computational modeling',
 'computer vision',
 'computing',
 'container technologies',
 'control theory',
 'critical thinking',
 'crm systems',
 'css'

In [7]:
final_test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/TalentFit/final_coverletter_df_test.csv')
